# Attempt to combine input of different shapes in sklearn
The model will have two inputs: one for time-dependent data (client purchases) and one for time-independent data (client data).

This time, I tried to combine xgboost for time-independent data and keras for time-dependent data by creating a tuple of datasets with different shapes.

In [39]:
import numpy as np
from xgboost import XGBClassifier
from sklift.models import ClassTransformation
from sklift import metrics
from sklift.viz import plot_qini_curve
from sklearn.pipeline import Pipeline
from sklearn.ensemble import VotingClassifier
import tensorflow as tf
from tensorflow import keras
from scikeras.wrappers import KerasClassifier
import pandas as pd

In [21]:
import chime

In [22]:
%load_ext chime

The chime extension is already loaded. To reload it, use:
  %reload_ext chime


In [35]:
class InputSelect():
    def __init__(self, n_input):
        self.n_input = n_input
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        print(X[self.n_input].shape)
        return X[self.n_input]
        


In [24]:
clf0 =  Pipeline([('select', InputSelect(0)), ('xgb', XGBClassifier())])

In [43]:
def get_model(hidden_layer_dim, meta):
    n_features_in_ = meta["n_features_in_"]
    X_shape_ = meta["X_shape_"]
    n_classes_ = meta["n_classes_"]

    model = keras.models.Sequential()
    model.add(keras.layers.Dense(n_features_in_, 
                                 input_shape=X_shape_[1:],
                                activation='relu'))
    model.add(keras.layers.Dense(hidden_layer_dim, activation='relu'))
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    return model

neural = KerasClassifier(
    get_model,
    loss="binary_crossentropy",
    hidden_layer_dim=4,
)



In [44]:
clf1 =  Pipeline([('select', InputSelect(1)), ('neural', neural)])

In [27]:
X0 = np.random.rand(5, 3)
X0

array([[0.03797125, 0.83718193, 0.02252683],
       [0.18087422, 0.48976587, 0.11048471],
       [0.35677924, 0.13801391, 0.10801287],
       [0.51201592, 0.0035916 , 0.07825074],
       [0.80042356, 0.434177  , 0.06981468]])

In [28]:
X1 = np.random.rand(5, 2, 4)

In [29]:
X = (X0, X1)

In [53]:
treat = pd.Series([0, 1, 0, 1, 0])

In [40]:
y = pd.Series([1, 1, 1, 0, 0])

In [47]:
clf = VotingClassifier(estimators=[('clf0', clf0), ('clf1', clf1)],
                      voting='soft')

In [51]:
model = ClassTransformation(estimator=clf)

In [54]:
model.fit(X, y, treat)

ValueError: Found input variables with inconsistent numbers of samples: [2, 5, 5]

# Does not work because the model takes n samples in X to be equal 2.